In [5]:
#dependencies

import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

### ENCODER PART
1. DATA LOADING
2. IMAGE PREPROCESSING, RESIZING, CLEANING
3. VISION TRANSFORMER ENCODER
*   patch embeddding
*   vision transformer
4. TRAINING LOOP

DATA LOADING OF POLYVORE SET
Do we want set to be categorized beforehand or not?

In [13]:
import os
directory = 'C:/Users/justin/Desktop/fash/Re-PolyVore'
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    print(file_path)
    if os.path.isfile(file_path):  # Check if it's a file
        print(file_path)

C:/Users/justin/Desktop/fash/Re-PolyVore\bag
C:/Users/justin/Desktop/fash/Re-PolyVore\bracelet
C:/Users/justin/Desktop/fash/Re-PolyVore\brooch
C:/Users/justin/Desktop/fash/Re-PolyVore\dress
C:/Users/justin/Desktop/fash/Re-PolyVore\earrings
C:/Users/justin/Desktop/fash/Re-PolyVore\eyewear
C:/Users/justin/Desktop/fash/Re-PolyVore\gloves
C:/Users/justin/Desktop/fash/Re-PolyVore\hairwear
C:/Users/justin/Desktop/fash/Re-PolyVore\hats
C:/Users/justin/Desktop/fash/Re-PolyVore\jumpsuit
C:/Users/justin/Desktop/fash/Re-PolyVore\legwear
C:/Users/justin/Desktop/fash/Re-PolyVore\necklace
C:/Users/justin/Desktop/fash/Re-PolyVore\neckwear
C:/Users/justin/Desktop/fash/Re-PolyVore\outwear
C:/Users/justin/Desktop/fash/Re-PolyVore\pants
C:/Users/justin/Desktop/fash/Re-PolyVore\rings
C:/Users/justin/Desktop/fash/Re-PolyVore\shoes
C:/Users/justin/Desktop/fash/Re-PolyVore\skirt
C:/Users/justin/Desktop/fash/Re-PolyVore\top
C:/Users/justin/Desktop/fash/Re-PolyVore\watches


IMAGE PREPROCESSING

Resizing down to 256x256 size, or maybe smaller, depending on training speed and quality

VISION TRANSFORMER ENCODER - supervised learning

In [7]:
class PatchEmbedding(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super(PatchEmbedding, self).__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        self.embed_dim = embed_dim

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x)  # Shape: (batch_size, embed_dim, num_patches ** 0.5, num_patches ** 0.5)
        x = x.flatten(2)  # Shape: (batch_size, embed_dim, num_patches)
        x = x.transpose(1, 2)  # Shape: (batch_size, num_patches, embed_dim)
        return x

class VisionTransformer(nn.Module):
    def __init__(self, img_size=224, patch_size=16, num_classes=1000, embed_dim=768, depth=12, num_heads=12):
        super(VisionTransformer, self).__init__()
        self.patch_embed = PatchEmbedding(img_size=img_size, patch_size=patch_size, embed_dim=embed_dim)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, self.patch_embed.num_patches + 1, embed_dim))
        self.pos_drop = nn.Dropout(p=0.1)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(embed_dim, num_heads), num_layers=depth
        )
        self.norm = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        x = self.patch_embed(x)
        batch_size = x.size(0)
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)
        x = self.transformer(x)
        x = self.norm(x[:, 0])
        x = self.head(x)
        return x

# Define the model
model = VisionTransformer(img_size=224, patch_size=16, num_classes=1000, embed_dim=768, depth=12, num_heads=12)


VISION TRANSFORMER ENCODER - self-supervised learning

A decoding part is required to reconstruct masked patches or perform other reconstruction tasks.

In [ ]:
# Modified Vision Transformer for self-supervised learning
class VisionTransformerSSL(nn.Module):
    def __init__(self, img_size=224, patch_size=16, embed_dim=768, depth=12, num_heads=12):
        super(VisionTransformerSSL, self).__init__()
        self.patch_embed = PatchEmbedding(img_size=img_size, patch_size=patch_size, embed_dim=embed_dim)
        self.pos_embed = nn.Parameter(torch.zeros(1, self.patch_embed.num_patches + 1, embed_dim))
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(embed_dim, num_heads), num_layers=depth
        )
        self.decoder = nn.Linear(embed_dim, patch_size * patch_size * 3)  # Assuming RGB patches

    def forward(self, x):
        x = self.patch_embed(x)
        batch_size = x.size(0)
        x = x + self.pos_embed[:, 1:, :]
        x = self.transformer(x)
        x = self.decoder(x)
        return x

# Define the model, loss function, and optimizer
model_ssl = VisionTransformerSSL(img_size=224, patch_size=16, embed_dim=768, depth=12, num_heads=12)
criterion = nn.MSELoss()
optimizer = optim.AdamW(model_ssl.parameters(), lr=3e-4)

# Dummy training loop for self-supervised learning
for epoch in range(num_epochs):
    model_ssl.train()
    running_loss = 0.0
    for inputs, _ in train_loader:
        inputs = inputs.to(device)
        # Create masked inputs and targets
        masked_inputs, targets = create_masked_patches(inputs)
        optimizer.zero_grad()
        outputs = model_ssl(masked_inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

VISION TRANSFORMER ENCODER - pre trained model

In [ ]:
from transformers import ViTModel, ViTFeatureExtractor
import torch

# Load pre-trained Vision Transformer model and feature extractor
model_name = "google/vit-base-patch16-224"
model = ViTModel.from_pretrained(model_name)
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)

# Define a function to extract features
def extract_vit_features(images):
    inputs = feature_extractor(images=images, return_tensors="pt")
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy()  # CLS token features

# Example usage with an image
from PIL import Image
image = Image.open('path_to_image.jpg')
features = extract_vit_features([image])
print(features.shape)  # Should be (1, 768) for ViT-base

TRAINING LOOP

In [ ]:
# Data preprocessing
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
train_dataset = datasets.ImageFolder(root='path_to_train_dataset', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

### RECOMMENDER PART

VECTOR EXTRACTION

In [ ]:
# Extracting features (latent vectors) from the trained Vision Transformer
def extract_features(model, dataloader):
    model.eval()
    features = []
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            outputs = model.patch_embed(inputs)  # Get patch embeddings
            cls_tokens = model.cls_token.expand(inputs.size(0), -1, -1)
            x = torch.cat((cls_tokens, outputs), dim=1)
            x = x + model.pos_embed
            x = model.pos_drop(x)
            x = model.transformer(x)
            x = model.norm(x[:, 0])
            features.append(x.cpu().numpy())
    return np.concatenate(features)

# Data loader for feature extraction
feature_loader = DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=4)
item_features = extract_features(model, feature_loader)

USER PROFILE AND RECOMMENDATION (currently ML, could be upgraded to a deep learning model based on user specifications)

In [ ]:
# Example: User interaction data
user_interactions = {
    'user1': [0, 1, 2],  # Indices of items the user interacted with
    'user2': [3, 4, 5],
    # ...
}

# Create user profiles based on average features of interacted items
user_profiles = {
    user_id: np.mean([item_features[item_id] for item_id in items], axis=0)
    for user_id, items in user_interactions.items()
}

# Recommendation function
from sklearn.metrics.pairwise import cosine_similarity

def recommend(user_id, top_n=10):
    user_vector = user_profiles[user_id].reshape(1, -1)
    similarities = cosine_similarity(user_vector, item_features)
    recommended_items = np.argsort(similarities[0])[-top_n:]
    return recommended_items

# Example usage
user_id = 'user1'
recommended_items = recommend(user_id)
print(f"Recommended items for user {user_id}: {recommended_items}")